In [1]:
import pandas as pd
import random as rnd
from docx import Document
from pathlib import Path

rows = 100
df = pd.read_csv("../Datasets/discharge.csv", nrows=rows)
df.to_json('../Datasets/test.json')

In [2]:
df

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...
...,...,...,...,...,...,...,...,...
95,10002430-DS-7,10002430,26295318,DS,7,2129-06-24 00:00:00,2129-06-24 16:51:00,\nName: ___ Unit No: ___\n \n...
96,10002443-DS-15,10002443,21329021,DS,15,2183-10-20 00:00:00,2183-10-20 19:18:00,\nName: ___ Unit No: ...
97,10002495-DS-13,10002495,24982426,DS,13,2141-05-29 00:00:00,2141-05-30 02:29:00,\nName: ___ Unit No: ___\...
98,10002528-DS-17,10002528,23193578,DS,17,2168-12-20 00:00:00,2168-12-20 17:28:00,\nName: ___ Unit No: ___\n...


In [3]:
test_list = df["subject_id"].unique()
print(test_list)

[10000032 10000084 10000117 10000248 10000560 10000764 10000826 10000883
 10000935 10000980 10001176 10001186 10001217 10001338 10001401 10001663
 10001667 10001725 10001860 10001877 10001884 10001919 10002013 10002131
 10002155 10002167 10002221 10002348 10002428 10002430 10002443 10002495
 10002528]


In [4]:
patient_list = [10000032, 10000935, 10001884]

patient_df = df[df['subject_id'].isin(patient_list)]
patient_df = patient_df.drop(['note_type', 'note_seq', 'storetime'], axis=1)
patient_df

,note_id,subject_id,hadm_id,charttime,text
0,10000032-DS-21,10000032,22595853,2180-05-07 00:00:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,2180-06-27 00:00:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,2180-07-25 00:00:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,2180-08-07 00:00:00,\nName: ___ Unit No: _...
14,10000935-DS-18,10000935,29541074,2183-11-04 00:00:00,\nName: ___ Unit No: ___...
15,10000935-DS-19,10000935,21738619,2187-07-12 00:00:00,\nName: ___ Unit No: ___...
16,10000935-DS-20,10000935,26381316,2187-08-27 00:00:00,\nName: ___ Unit No: ___...
17,10000935-DS-21,10000935,25849114,2187-10-26 00:00:00,\nName: ___ Unit No: ___...
46,10001884-DS-24,10001884,21268656,2125-10-20 00:00:00,\nName: ___ Unit No: ___\n \nA...
47,10001884-DS-25,10001884,26679629,2125-10-27 00:00:00,\nName: ___ Unit No: ___\n \nA...


In [7]:
random = rnd.randint(1,len(patient_df))

test_cell = patient_df.iloc[random, 4]    

print(f"{random}, {test_cell}")

17,  
Name:  ___             Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
IV Dye, Iodine Containing Contrast Media / Oxycodone / 
cilostazol / Varenicline
 
Attending: ___.
 
Chief Complaint:
cough, dyspnea
 
Major Surgical or Invasive Procedure:
None

 
History of Present Illness:
HPI: ___ year old female with history of COPD (on home O2), HTN, 
Afib admitted with dyspnea and cough.  
 Pt states inc dyspnea since this am, also one episode of 
retrosternal chest pressure lasting 2minuts on way to ED. No cp 
currently. on home O2. no fevers/chills or abd sx.  
 Patient was recently admitted from ___ with COPD flare 
and afib with RVR. She could not receive azithromycin due to 
concern for QTc prolongation and so was treated with 
ceftriaxone/cefpodoxime. She was treated with 60mg PO prednisone 
and discharged with a prednisone taper of 10 mg decrease q3d 
until at 10 mg, then stay at

In [8]:
DATA = patient_df
# Output folder
out_dir = Path('../Documents')
out_dir.mkdir(exist_ok=True)

def safe_name(s):
    return "".join(ch for ch in str(s) if ch not in r'\/:*?"<>|').strip() or "unnamed"

created = 0
for _, row in DATA.iterrows():
    doc = Document()

    # Header
    doc.add_heading(f"Note ID: {row['note_id']}", level=1)

    # Metadata
    p = doc.add_paragraph()
    p.add_run("Subject ID: ").bold = True
    p.add_run(str(row['subject_id']))
    p = doc.add_paragraph()
    p.add_run("HADM ID: ").bold = True
    p.add_run(str(row['hadm_id']))

    doc.add_paragraph("")  # spacer
    doc.add_heading("Text", level=2)
    doc.add_paragraph("" if pd.isna(row['text']) else str(row['text']))

    # Filename from note_id
    fname = out_dir / f"note_{safe_name(row['note_id'])}.docx"
    doc.save(fname)
    created += 1

print(f"✅ Created {created} documents in {out_dir.resolve()}")

✅ Created 23 documents in /Users/maui/Developer/Developer Projects/Python/Text-Analytics-Final-Project/Documents
